In [1]:
import sys
sys.path.append("../")
import pandas as pd
import plotly.graph_objects as go
import datetime as dt
from plotting import CandlePlot

In [2]:
from infrastructure.instrument_collection import instrumentCollection as ic

In [32]:
pair = "GBP_JPY"
granularity = "H4"
df = pd.read_pickle(f"../data/{pair}_{granularity}.pkl")
MA_LIST = [10, 20, 50, 100, 200]

In [33]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [34]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c
0,2021-01-07 06:00:00+00:00,140.206,140.853,140.188,140.792
1,2021-01-07 10:00:00+00:00,140.790,140.994,140.618,140.806
2,2021-01-07 14:00:00+00:00,140.802,140.996,140.604,140.746
3,2021-01-07 18:00:00+00:00,140.749,140.992,140.749,140.842
4,2021-01-07 22:00:00+00:00,140.820,140.914,140.743,140.808


In [35]:
for ma in MA_LIST:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
    df_ma.dropna(inplace=True)
    df_ma.reset_index(inplace=True, drop=True)


In [36]:
# df_ma['MA_20'] = df_ma.mid_c.rolling(window=20).mean()

In [37]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_10,MA_20,MA_50,MA_100,MA_200
0,2021-04-05 17:00:00+00:00,153.060,153.280,153.048,153.192,153.0735,152.92065,151.68162,151.41180,150.360270
1,2021-04-05 21:00:00+00:00,153.136,153.374,153.052,153.306,153.1315,152.95425,151.76220,151.42685,150.393850
2,2021-04-06 01:00:00+00:00,153.303,153.416,153.206,153.316,153.1617,152.99115,151.85268,151.44533,150.426575
3,2021-04-06 05:00:00+00:00,153.314,153.386,152.844,152.952,153.1567,153.00065,151.92904,151.45683,150.458185
4,2021-04-06 09:00:00+00:00,152.954,153.056,152.084,152.110,153.0541,152.98605,151.98712,151.46017,150.485435


In [38]:
df_plot = df_ma.iloc[:500]

In [39]:
cp = CandlePlot(df_plot)

In [40]:
traces = [ f"MA_{x}" for x in MA_LIST ]

In [41]:
cp.show_plot(line_traces=traces)

In [61]:
MA_S = "MA_50"
MA_L = "MA_200"
BUY = 1
SELL = -1
NONE = 0

In [62]:
df_an = df_ma[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', MA_S, MA_L]].copy()

In [63]:
df_an.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200
0,2021-04-05 17:00:00+00:00,153.060,153.280,153.048,153.192,151.68162,150.360270
1,2021-04-05 21:00:00+00:00,153.136,153.374,153.052,153.306,151.76220,150.393850
2,2021-04-06 01:00:00+00:00,153.303,153.416,153.206,153.316,151.85268,150.426575
3,2021-04-06 05:00:00+00:00,153.314,153.386,152.844,152.952,151.92904,150.458185
4,2021-04-06 09:00:00+00:00,152.954,153.056,152.084,152.110,151.98712,150.485435


In [64]:
df_an['DELTA'] = df_an.MA_50 - df_an.MA_200
df_an['DELTA_PREV'] = df_an.DELTA.shift(1)

In [65]:
def is_trade(row):
    if row.DELTA >= 0 and row.DELTA_PREV < 0: 
        return BUY
    if row.DELTA < 0 and row.DELTA_PREV >= 0:
        return SELL
    return NONE

In [66]:
df_an['TRADE'] = df_an.apply(is_trade, axis = 1)

In [67]:
df_trades = df_an[df_an.TRADE != NONE].copy()

In [68]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200,DELTA,DELTA_PREV,TRADE
47,2021-04-15 13:00:00+00:00,149.988,149.994,149.690,149.856,150.82828,150.902175,-0.073895,0.000280,-1
127,2021-05-04 21:00:00+00:00,151.845,152.005,151.760,151.959,150.96454,150.949005,0.015535,-0.027730,1
331,2021-06-21 21:00:00+00:00,153.628,153.776,153.306,153.411,154.30770,154.323890,-0.016190,0.017955,-1
541,2021-08-09 21:00:00+00:00,152.768,152.905,152.712,152.815,152.50896,152.508625,0.000335,-0.010650,1
583,2021-08-18 21:00:00+00:00,150.986,151.139,150.946,151.051,152.14970,152.150745,-0.001045,0.023490,-1


In [69]:
cp = CandlePlot(df_an.iloc[40 : 100])
cp.show_plot(line_traces = [MA_S, MA_L])

In [70]:
ic.LoadInstruments("../data")

In [71]:
ins_data = ic.instruments_dict[pair]

In [72]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200,DELTA,DELTA_PREV,TRADE
47,2021-04-15 13:00:00+00:00,149.988,149.994,149.690,149.856,150.82828,150.902175,-0.073895,0.000280,-1
127,2021-05-04 21:00:00+00:00,151.845,152.005,151.760,151.959,150.96454,150.949005,0.015535,-0.027730,1
331,2021-06-21 21:00:00+00:00,153.628,153.776,153.306,153.411,154.30770,154.323890,-0.016190,0.017955,-1
541,2021-08-09 21:00:00+00:00,152.768,152.905,152.712,152.815,152.50896,152.508625,0.000335,-0.010650,1
583,2021-08-18 21:00:00+00:00,150.986,151.139,150.946,151.051,152.14970,152.150745,-0.001045,0.023490,-1


In [73]:
df_trades['DIFF'] = df_trades.mid_c.diff().shift(-1)
df_trades.fillna(0, inplace=True)

In [74]:
df_trades['GAIN'] = df_trades['DIFF'] / ins_data.pipLocation
df_trades['GAIN'] = df_trades["GAIN"] * df_trades['TRADE']

In [75]:
df_trades.GAIN.sum()

-6498.20000000001

In [76]:
df_trades['GAIN_C'] = df_trades['GAIN'].cumsum()

In [77]:
cp = CandlePlot(df_trades, candles=False)
cp.show_plot(line_traces=['GAIN_C'])